In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import geopandas as gpd
gpd.read_file('/home/anton/dima_experiments/Vip_gZ/data/initial_gauges/great_db/recalc_13_03_2023.gpkg')

,gauge_id,name_ru,name_en,area_db,checked,x,y,merge,ais_area,reason,recalc,area,layer,path,geometry
0,6128,р.Хила - ст.Ага,r.Hila - st.Aga,1480.0,1.0,115.17468,51.182243,None,1480.0,бессточные области,0.0,NaN,discharge_gauges_13_03_2023_recalc,/home/dima/HSI_server/dima_experiments/Vip_gZ/...,POINT (114.88765 50.91113)
1,6073,р.Шилка - ст.Часовая,r.Shilka - st.Chasovaja,NaN,NaN,NaN,NaN,None,NaN,None,NaN,200000.0,output,/home/dima/HSI_server/dima_experiments/Vip_gZ/...,POINT (119.97305 53.41355)
2,3321,р.Вилюй - с.Сунтар,r.Viljuj - s.Suntar,NaN,NaN,NaN,NaN,None,NaN,None,NaN,202000.0,output,/home/dima/HSI_server/dima_experiments/Vip_gZ/...,POINT (117.65000 62.15000)


In [ ]:
import geopandas as gpd
import glob
from pathlib import Path
from tqdm.notebook import tqdm

from src.geom_functions import (ws_AOI, select_big_from_MP)
from src.data_preparation import aoi_tiles
from src.grid_calc import dir_acc_for_aoi




In [3]:
mask_files = {file.split('/')[-1][:-4]: select_big_from_MP(gpd.read_file(
    file, encoding='utf-8').loc[0, 'geometry'])
    for file in glob.glob('./data/ws_masks/*.shp')}

path_to_masks = Path('./data/region_aois/')
path_to_masks.mkdir(exist_ok=True, parents=True)
masks_aoi = {region: ws_AOI(ws=ws,
                            shp_path=f'{path_to_masks}/{region}_aoi.shp')
             for region, ws in mask_files.items()}




In [54]:
for region, aoi_file in tqdm(masks_aoi.items()):

    tiles = aoi_tiles(topo_p='../LISFLOOD/data/topography/',
                      aoi_shp=aoi_file)['dir']

    dir_acc_for_aoi(region_name=region,
                    tiles=tiles,
                    result_folder='./data/aois_rasters/')


In [6]:
aoi_file = masks_aoi['south']
tiles = aoi_tiles(topo_p='../LISFLOOD/data/topography/',
                    aoi_shp=aoi_file)['dir']

dir_acc_for_aoi(region_name='south',
                tiles=tiles,
                result_folder='./data/aois_rasters/')

0...10...20...30...40...50...60...70...80...90...100 - done.

south полностью обсчитан !



In [13]:
from shapely import ops, geometry
import gc
from copy import deepcopy
from src.geom_functions import polygon_area
from pysheds.grid import Grid
from pysheds.pview import Raster


def my_catchment(grid_p: str,
                 fdir_p: str,
                 facc_p: str,
                 gauges_file,
                 save_p: str,
                 region_name: str):
    ws = None

    for i in range(len(gauges_file)):

        grid = Grid.from_raster(grid_p, data_name='fdir_grid',
                                nodata=0)
        fdir = grid.read_raster(fdir_p, data_name='fdir',
                                nodata=0)
        acc = grid.read_raster(facc_p, data_name='acc',
                               nodata=0)

        name = gauges_file.loc[i, 'name']
        area = gauges_file.loc[i, 'area']

        if area is None:
            print(f'Calculation for {name} -- Area, sq.km = {area}')
        else:
            print(f'\nCalculation for {name} -- Area, sq.km = {area:.2f}')
           
        x_coord = gauges_file.loc[i, 'x']
        y_coord = gauges_file.loc[i, 'y']
        # Determine D8 flow directions from DEM
        # ----------------------
        # Specify directional mapping
        dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
        # Snap pour point to high accumulation cell
        if isinstance(acc, Raster):
            x_snap, y_snap = grid.snap_to_mask(acc > 1e3, (x_coord, y_coord))
            del acc
            gc.collect()
            # Delineate the catchment
            catch = grid.catchment(x=x_snap, y=y_snap,
                                data=fdir,
                                dirmap=dirmap,
                                xytype='coordinate')
            # Crop and plot the catchment
            # ---------------------------
            # Clip the bounding box to the catchment
            grid.clip_to(catch)
            # clipped_catch = initial_grid.view(ws_grid)

            ws = grid.polygonize()
            ws = ops.unary_union([geometry.shape(shape)
                                  for shape, _ in ws])
            ws = select_big_from_MP(ws)
            del grid
            del fdir
            gc.collect()

    gauges_file = gpd.GeoDataFrame(gauges_file).set_crs(epsg=4326)
    gauges_file['area_dif'] = (
        gauges_file['area'] - gauges_file['new_area']) \
        / (gauges_file['area']) * 100

    gauges_file.to_file(f'{save_p}/{region_name}.shp',
                        encoding='utf-8')

In [12]:
f_path = './data/kuban/gauges_kuban.gpkg'

test_gauge = gpd.read_file(f_path, encoding='utf-8')
test_gauge = read_gauge(test_gauge)

dir_tiff = './data/aois_rasters/fdir/south_dir.tif'
acc_tiff = './data/aois_rasters/acc/south_acc.tiff'


for i in range(len(test_gauge)):
    name = test_gauge.loc[i, 'name']
    if test_gauge.loc[i, 'area'] is not None:
        area = float(test_gauge.loc[i, 'area'])
        print(f'\nCalculation for {name} -- Area, sq.km = {area:.2f}')
    else:
        area = None
        print(f'Calculation for {name} -- Area, sq.km = {area}')
    x_coord = test_gauge.loc[i, 'x']
    y_coord = test_gauge.loc[i, 'y']

    test_gauge.loc[i, 'geometry'] = my_catchment(
        grid=deepcopy(grid),
        fdir=fdir,
        acc=acc,
        x=x_coord,
        y=y_coord)
    new_area = polygon_area(
        lats=test_gauge.loc[i, 'geometry'].exterior.coords.xy[1],
        lons=test_gauge.loc[i, 'geometry'].exterior.coords.xy[0])
    test_gauge.loc[i, 'new_area'] = new_area

    print(f'Calculated area, sq.km {new_area:.2f}')


,id,code,code_1,name,y,x,ugms,open,close,date_open,date_close,basin,code_vod,name_vod,area,discharge,geometry,point_geom
0,1,1109,1109,руч.Кривуля - устье,44.62070,39.40630,Колымское УГМС,01.01.1942,01.01.1998,01.01.1942,01.01.1998,Бассейн Восточно-Сибирского моря,119000492,"(руч. Кривуля, Безымянный)",8.2,1,None,POINT (39.4063 44.6207)
1,2,83132,83132,р.Кубань - пос.Худес,43.59600,42.13450,Северо-Кавказское УГМС,15.12.1965,28.03.1969,15.12.1965,28.03.1969,Бассейн р. Кубани,108100109,Кубань (Петрушин рук.),1540,0,None,POINT (42.1345 43.596)
2,3,83133,83133,р.Кубань - г.Микоян-Шахар,44.40050,41.94040,Северо-Кавказское УГМС,25.08.1923,30.04.1939,25.08.1923,30.04.1939,Бассейн р. Кубани,108100109,Кубань (Петрушин рук.),2480,0,None,POINT (41.9404 44.4005)
3,4,83134,83134,р.Кубань - г.Карачаевск,43.78670,41.91520,Северо-Кавказское УГМС,01.12.1954,29.12.1967,01.12.1954,29.12.1967,Бассейн р. Кубани,108100109,Кубань (Петрушин рук.),2480,1,None,POINT (41.9152 43.7867)
4,5,83137,83137,р.Кубань - с.им.Коста-Хетагурова,43.81037,41.90181,Северо-Кавказское УГМС,03.09.1921,действующий,03.09.1921,31.12.2021,Бассейн р. Кубани,108100109,Кубань (Петрушин рук.),3800,1,None,POINT (41.90181 43.81037)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,175,83812,83812,"р.Кубань,рук.Петрушин - Переволокский Узел",45.22930,37.37210,Северо-Кавказское УГМС,14.08.1915,14.03.1942,14.08.1915,14.03.1942,Выпуск 1,108100648,"Кубань, рук. Петрушин",None,1,None,POINT (37.3721 45.2293)
175,176,83813,83813,"р.Кубань,рук.Петрушин - г.Темрюк",45.28901,37.36195,Северо-Кавказское УГМС,14.03.1911,действующий,14.03.1911,31.12.2021,Бассейн р. Кубани,108100109,Кубань (Петрушин рук.),None,1,None,POINT (37.36195 45.28901)
176,177,83814,83814,р.Кубань - с.Вербино,45.33230,37.40170,Северо-Кавказское УГМС,16.07.1957,27.10.1969,16.07.1957,27.10.1969,Бассейн р. Кубани,108100109,Кубань (Петрушин рук.),None,0,None,POINT (37.4017 45.3323)
177,178,83831,83831,рук.Переволока - Переволокский Узел,45.23070,37.36750,Северо-Кавказское УГМС,01.01.1915,30.06.1934,01.01.1915,30.06.1934,Нет данных о бассейне,0,нет данных,None,0,None,POINT (37.3675 45.2307)
